# Scientometric Analysis for DoD Lean Six Sigma Research

## Setup

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import networkx as nx
from wordcloud import WordCloud

import openpyxl
from openpyxl.styles import Font, PatternFill
from openpyxl.utils import get_column_letter

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import os


In [3]:
# Set up the environment
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("colorblind")

# Create results directory
RESULTS_PATH = "results"
os.makedirs(RESULTS_PATH, exist_ok=True)

# Load the data
print("Loading data...")
df = pd.read_excel("sources.xlsx")

Loading data...


In [4]:
# Download necessary NLTK resources (uncomment if needed)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Functions

In [5]:
def analyze_publication_trend(df):
    """Create publication trend line figure"""
    print("Analyzing publication trends...")
    
    # Group by year and count publications
    publications_by_year = df['year'].value_counts().sort_index()
    years = list(publications_by_year.index)
    publication_counts = list(publications_by_year.values)
    
    # Create publication trend visualization
    plt.figure(figsize=(12, 6))
    plt.bar(years, publication_counts, color='steelblue', alpha=0.7)
    plt.plot(years, publication_counts, color='red', marker='o', linewidth=2)
    
    # Add trend line
    z = np.polyfit(years, publication_counts, 1)
    p = np.poly1d(z)
    plt.plot(years, p(years), "r--", alpha=0.8, linewidth=1.5)
    
    # Add 3-year moving average
    if len(years) >= 3:
        moving_avg = []
        for i in range(len(publication_counts) - 2):
            avg = np.mean(publication_counts[i:i+3])
            moving_avg.append(avg)
        
        plt.plot(years[1:-1], moving_avg, color='green', marker='x', 
                 linestyle='-.', linewidth=1.5, label='3-Year Moving Average')
        plt.legend()
    
    plt.title('Publication Trend of DoD Lean Six Sigma Research', fontsize=16)
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Number of Publications', fontsize=14)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(years, rotation=45)
    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/publication_trend.png', dpi=300)
    plt.close()
    
    # Export data to CSV
    trend_df = pd.DataFrame({'Year': years, 'Publications': publication_counts})
    trend_df.to_csv(f'{RESULTS_PATH}/publication_trend_data.csv', index=False)

In [6]:
def preprocess_text(text):
    """Preprocess text for NLP analysis"""
    if pd.isna(text):
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords and short words
    stop_words = set(stopwords.words('english'))
    custom_stopwords = {'department', 'defense', 'dod', 'military', 'study', 'research', 
                      'paper', 'analysis', 'approach', 'method', 'methodology',
                      'data', 'result', 'thesis', 'dissertation', 'university'}
    stop_words.update(custom_stopwords)
    
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words and len(word) > 2]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return ' '.join(tokens)

In [7]:
def analyze_document_types(df):
    """Analyze document types in the dataset using automatic categorization"""
    print("Analyzing document types...")
    
    # Check if we have the document type column
    if 'documentType' not in df.columns:
        print("No 'documentType' column found. Checking alternate column names...")
        
        # Try alternate column names
        alt_names = ['DocumentType', 'document_type', 'articleType', 'ArticleType', 'docType', 'DocType']
        found = False
        
        for col in alt_names:
            if col in df.columns:
                print(f"Found document type information in column '{col}'")
                df['documentType'] = df[col]
                found = True
                break
        
        if not found:
            print("No document type information found in the dataset.")
            return None
    
    # Clean document types (handle missing values)
    df['documentType'] = df['documentType'].fillna('Unknown')
    
    # Extract unique document types automatically
    unique_doc_types = set()
    
    # Handle compound document types (e.g., "Journal Article, Case Study")
    all_doc_types = []
    for doc_type in df['documentType']:
        if pd.isna(doc_type) or doc_type == '':
            continue
            
        # Split by comma if it contains multiple types
        if ',' in doc_type:
            types = [t.strip() for t in doc_type.split(',')]
            for t in types:
                if t:  # Only add non-empty strings
                    unique_doc_types.add(t)
                    all_doc_types.append(t)
        else:
            if doc_type.strip():  # Only add non-empty strings
                unique_doc_types.add(doc_type.strip())
                all_doc_types.append(doc_type.strip())
    
    print(f"Found {len(unique_doc_types)} unique document types:")
    for doc_type in sorted(unique_doc_types):
        print(f"  - {doc_type}")
    
    # Count document types
    doc_type_counts = pd.Series(all_doc_types).value_counts()
    
    # Remove 'Unknown' if it's a small percentage
    if 'Unknown' in doc_type_counts and doc_type_counts['Unknown'] / doc_type_counts.sum() < 0.05:
        doc_type_counts = doc_type_counts.drop('Unknown')
    
    # Calculate percentage distribution
    doc_type_percent = (doc_type_counts / doc_type_counts.sum() * 100).round(1)
    
    # Function to assign primary document type (for papers with multiple types)
    def get_primary_doc_type(doc_type):
        if pd.isna(doc_type) or doc_type == '':
            return 'Unknown'
            
        # If it has multiple types, take the first one
        if ',' in doc_type:
            return doc_type.split(',')[0].strip()
        
        return doc_type.strip()
    
    # Assign primary document type to each paper
    df['primary_doc_type'] = df['documentType'].apply(get_primary_doc_type)
    
    # Create visualizations only if we have enough unique types
    if len(doc_type_counts) > 0:
        # Create pie chart - limit to top 10 categories if there are too many
        if len(doc_type_counts) > 10:
            print(f"Limiting pie chart to top 10 of {len(doc_type_counts)} document types")
            plot_counts = doc_type_counts.nlargest(9)
            # Add an "Other" category for the rest
            other_count = doc_type_counts[~doc_type_counts.index.isin(plot_counts.index)].sum()
            plot_counts['Other'] = other_count
        else:
            plot_counts = doc_type_counts
        
        plt.figure(figsize=(12, 8))
        wedges, texts, autotexts = plt.pie(
            plot_counts, 
            labels=plot_counts.index, 
            autopct='%1.1f%%',
            shadow=False, 
            startangle=90, 
            explode=[0.05] * len(plot_counts),
            colors=plt.cm.tab10(np.linspace(0, 1, len(plot_counts)))
        )
        
        # Enhance text visibility
        for text in texts:
            text.set_fontsize(11)
        for autotext in autotexts:
            autotext.set_fontsize(9)
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        plt.title('Distribution of Document Types in DoD Lean Six Sigma Literature', fontsize=16)
        plt.axis('equal')
        plt.tight_layout()
        plt.savefig(f'{RESULTS_PATH}/document_type_distribution_pie.png', dpi=300)
        plt.close()
        
        # Create bar chart with count and percentage
        plt.figure(figsize=(14, 8))
        
        # For bar chart, show all categories but limit if there are too many
        if len(doc_type_counts) > 15:
            plot_counts = doc_type_counts.nlargest(15)
            plt.title('Top 15 Document Types in DoD Lean Six Sigma Literature', fontsize=16)
        else:
            plot_counts = doc_type_counts
            plt.title('Distribution of Document Types in DoD Lean Six Sigma Literature', fontsize=16)
        
        bars = plt.bar(
            plot_counts.index,
            plot_counts.values,
            color=plt.cm.tab10(np.linspace(0, 1, len(plot_counts)))
        )
        
        plt.xlabel('Document Type', fontsize=14)
        plt.ylabel('Number of Documents', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Add data labels with count and percentage
        for i, (count, percent) in enumerate(zip(plot_counts, 
                                                [doc_type_percent[idx] for idx in plot_counts.index])):
            plt.text(
                i, 
                count + 0.5, 
                f"{count} ({percent}%)", 
                ha='center', 
                fontsize=10,
                fontweight='bold'
            )
        
        plt.tight_layout()
        plt.savefig(f'{RESULTS_PATH}/document_type_distribution_bar.png', dpi=300)
        plt.close()
        
        # Document type trends over time
        try:
            # Group by year and document type (using primary doc type for trend analysis)
            yearly_doc_type = df.groupby(['year', 'primary_doc_type']).size().unstack(fill_value=0)
            
            # Only keep the most common document types for clarity
            if len(doc_type_counts) > 5:
                top_doc_types = doc_type_counts.nlargest(5).index.tolist()
                yearly_doc_type_filtered = yearly_doc_type[yearly_doc_type.columns.intersection(top_doc_types)]
                
                # If we don't have all top document types in the filtered DataFrame, add them with zeros
                for doc_type in top_doc_types:
                    if doc_type not in yearly_doc_type_filtered.columns:
                        yearly_doc_type_filtered[doc_type] = 0
                
                # Ensure we only have the top document types (no more, no less)
                yearly_doc_type_filtered = yearly_doc_type_filtered[
                    [col for col in top_doc_types if col in yearly_doc_type_filtered.columns]
                ]
            else:
                yearly_doc_type_filtered = yearly_doc_type
            
            if not yearly_doc_type_filtered.empty:
                # Plot document type trends over time
                plt.figure(figsize=(14, 8))
                ax = yearly_doc_type_filtered.plot(
                    kind='line', 
                    marker='o',
                    linewidth=2,
                    markersize=6,
                    ax=plt.gca()
                )
                
                plt.title('Document Types in DoD Lean Six Sigma Research Over Time', fontsize=16)
                plt.xlabel('Year', fontsize=14)
                plt.ylabel('Number of Publications', fontsize=14)
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.legend(title='Document Type', bbox_to_anchor=(1.05, 1), loc='upper left')
                plt.tight_layout()
                plt.savefig(f'{RESULTS_PATH}/document_type_trends.png', dpi=300)
                plt.close()
                
                # Create stacked area chart
                plt.figure(figsize=(14, 8))
                yearly_doc_type_filtered.plot.area(stacked=True, alpha=0.7, ax=plt.gca())
                plt.title('Cumulative Document Types in DoD Lean Six Sigma Research', fontsize=16)
                plt.xlabel('Year', fontsize=14)
                plt.ylabel('Number of Publications', fontsize=14)
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.legend(title='Document Type', bbox_to_anchor=(1.05, 1), loc='upper left')
                plt.tight_layout()
                plt.savefig(f'{RESULTS_PATH}/document_type_cumulative.png', dpi=300)
                plt.close()
                
                # Calculate proportional representation over time (percentage)
                yearly_doc_type_pct = yearly_doc_type_filtered.div(yearly_doc_type_filtered.sum(axis=1), axis=0) * 100
                
                plt.figure(figsize=(14, 8))
                yearly_doc_type_pct.plot.area(stacked=True, alpha=0.7, ax=plt.gca())
                plt.title('Proportion of Document Types in DoD Lean Six Sigma Research Over Time', fontsize=16)
                plt.xlabel('Year', fontsize=14)
                plt.ylabel('Percentage of Publications', fontsize=14)
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.legend(title='Document Type', bbox_to_anchor=(1.05, 1), loc='upper left')
                plt.tight_layout()
                plt.savefig(f'{RESULTS_PATH}/document_type_percentage.png', dpi=300)
                plt.close()
        except Exception as e:
            print(f"Error creating time trend visualizations: {e}")
    
    # Export document type data
    doc_type_df = pd.DataFrame({
        'Document_Type': doc_type_counts.index, 
        'Count': doc_type_counts.values,
        'Percentage': doc_type_percent.values
    })
    doc_type_df.to_csv(f'{RESULTS_PATH}/document_type_distribution.csv', index=False)
    
    try:
        if 'yearly_doc_type' in locals() and not yearly_doc_type.empty:
            yearly_doc_type.to_csv(f'{RESULTS_PATH}/document_type_yearly.csv')
    except:
        pass
    
    print(f"Document type analysis complete. Found {len(doc_type_counts)} document types.")
    
    return doc_type_df

In [8]:
def analyze_keywords(df):
    """Analyze keywords: frequency and network"""
    print("Analyzing keywords...")
    
    # Preprocess abstracts
    df['processed_abstract'] = df['Abstract'].apply(preprocess_text)
    
    # Combine all processed abstracts for word frequency analysis
    all_text = ' '.join(df['processed_abstract'].dropna())
    words = word_tokenize(all_text)
    word_freq = Counter(words)
    most_common_words = word_freq.most_common(30)
    
    # Plot word frequency
    words, counts = zip(*most_common_words)
    plt.figure(figsize=(12, 8))
    y_pos = np.arange(len(words))
    plt.barh(y_pos, counts, align='center', color='skyblue')
    plt.yticks(y_pos, words)
    plt.xlabel('Frequency')
    plt.title('Most Common Keywords in DoD Lean Six Sigma Literature', fontsize=16)
    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/keyword_frequency.png', dpi=300)
    plt.close()
    
    # Create word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white',
                          max_words=100, contour_width=3, contour_color='steelblue')
    wordcloud.generate(all_text)
    plt.figure(figsize=(16, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/wordcloud.png', dpi=300)
    plt.close()
    
    # Keyword co-occurrence analysis
    def extract_keywords(text):
        """Extract keywords from text"""
        if pd.isna(text):
            return []
        
        # Convert to lowercase
        text = text.lower()
        
        # Tokenize
        tokens = word_tokenize(text)
        
        # Remove stopwords and short words
        stop_words = set(stopwords.words('english'))
        custom_stopwords = {'department', 'defense', 'dod', 'military', 'study', 'research', 
                          'paper', 'analysis', 'approach', 'method', 'methodology'}
        stop_words.update(custom_stopwords)
        
        tokens = [word for word in tokens if word.isalpha() and word not in stop_words and len(word) > 2]
        return tokens
    
    # Extract keywords from each abstract
    df['keywords'] = df['Abstract'].apply(extract_keywords)
    
    # Create co-occurrence matrix
    keywords_list = []
    for keywords in df['keywords']:
        keywords_list.extend(keywords)
    
    # Get the most common keywords - limiting to top 30 instead of 50 for clearer visualization
    most_common = Counter(keywords_list).most_common(30)
    top_keywords = [word for word, count in most_common]
    
    # Create co-occurrence matrix
    co_occurrence = np.zeros((len(top_keywords), len(top_keywords)))
    for keywords in df['keywords']:
        for i, keyword1 in enumerate(top_keywords):
            if keyword1 in keywords:
                for j, keyword2 in enumerate(top_keywords):
                    if keyword2 in keywords:
                        co_occurrence[i, j] += 1
    
    # Create network but with stronger filtering for a less dense graph
    G = nx.Graph()
    
    # Add nodes first
    for i, keyword1 in enumerate(top_keywords):
        # Only add nodes with sufficient frequency
        if most_common[i][1] > 5:  # Only include keywords that appear more than 5 times
            G.add_node(keyword1, size=most_common[i][1])
    
    # Now add edges, but with a higher threshold
    threshold = max(5, np.mean(co_occurrence) + 0.5 * np.std(co_occurrence))  # Dynamic threshold
    print(f"Co-occurrence threshold: {threshold}")
    
    for i, keyword1 in enumerate(top_keywords):
        if keyword1 not in G:
            continue
        for j, keyword2 in enumerate(top_keywords):
            if i != j and keyword2 in G and co_occurrence[i, j] > threshold:
                G.add_edge(keyword1, keyword2, weight=co_occurrence[i, j])
    
    # Remove isolated nodes (no connections)
    isolated_nodes = [node for node, degree in dict(G.degree()).items() if degree == 0]
    G.remove_nodes_from(isolated_nodes)
    
    # Print network summary
    print(f"Network contains {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    
    # Check if we have a graph to visualize
    if G.number_of_nodes() == 0:
        print("No nodes in graph after filtering. Try reducing the threshold.")
        return
    
    # Create main network visualization
    plt.figure(figsize=(16, 16))
    
    # Try different layout algorithms for better visualization
    try:
        # Try community-based layout if we have enough nodes
        if G.number_of_nodes() > 5:
            pos = nx.spring_layout(G, k=0.4, iterations=100, seed=42)
        else:
            pos = nx.circular_layout(G)
    except:
        # Fallback to basic layout
        pos = nx.spring_layout(G, k=0.3, seed=42)
    
    # Calculate node sizes based on frequency, with normalization
    max_node_size = 2000
    min_node_size = 300
    node_sizes = []
    
    for node in G.nodes():
        size = G.nodes[node]['size']
        # Normalize to a reasonable size range
        normalized_size = min_node_size + (size / max([G.nodes[n]['size'] for n in G.nodes()])) * (max_node_size - min_node_size)
        node_sizes.append(normalized_size)
    
    # Calculate edge weights, normalized for better visualization
    max_width = 5
    min_width = 0.5
    
    if G.number_of_edges() > 0:
        max_weight = max([G.edges[edge]['weight'] for edge in G.edges()])
        edge_weights = [min_width + (G.edges[edge]['weight'] / max_weight) * (max_width - min_width) for edge in G.edges()]
    else:
        edge_weights = []
    
    # Draw the network with coloring based on centrality
    # Calculate betweenness centrality
    if G.number_of_nodes() > 1:
        centrality = nx.betweenness_centrality(G)
        centrality_values = [centrality[node] for node in G.nodes()]
        
        # Draw nodes with centrality-based colors
        nodes = nx.draw_networkx_nodes(G, pos, 
                                       node_size=node_sizes, 
                                       node_color=centrality_values,
                                       cmap=plt.cm.viridis, 
                                       alpha=0.8)
        
        # Add colorbar for centrality
        plt.colorbar(nodes, label='Betweenness Centrality', shrink=0.8)
    else:
        # Simple drawing for small networks
        nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color='skyblue', alpha=0.8)
    
    # Draw edges with transparency for less visual clutter
    nx.draw_networkx_edges(G, pos, width=edge_weights, alpha=0.3, edge_color='gray')
    
    # Draw labels with appropriate font size - FIX: Use a single font size value
    # Calculate the font size based on node importance, but as a number not a dictionary
    base_font_size = 10
    # Just use a fixed font size for all labels
    nx.draw_networkx_labels(G, pos, font_size=base_font_size, font_family='sans-serif', font_weight='bold')
    
    plt.title('Keyword Co-occurrence Network (Filtered for Clarity)', fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/keyword_network.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a simplified network visualization for better readability
    # Keep only the top connections
    G_simple = nx.Graph()
    
    # Add the top nodes
    top_n_keywords = 15  # Limit to top 15 keywords
    for i, (word, count) in enumerate(most_common[:top_n_keywords]):
        G_simple.add_node(word, size=count)
    
    # Add only the strongest connections
    high_threshold = max(10, np.percentile(co_occurrence[co_occurrence > 0], 90))  # Very high threshold
    
    for i, keyword1 in enumerate(top_keywords[:top_n_keywords]):
        for j, keyword2 in enumerate(top_keywords[:top_n_keywords]):
            if i < j and co_occurrence[i, j] > high_threshold:
                G_simple.add_edge(keyword1, keyword2, weight=co_occurrence[i, j])
    
    # Create a simplified visualization
    plt.figure(figsize=(12, 12))
    pos_simple = nx.spring_layout(G_simple, k=0.5, iterations=100, seed=42)
    
    # Node sizes simplified
    node_sizes_simple = [G_simple.nodes[node]['size'] * 30 for node in G_simple.nodes()]
    
    # Draw simplified network
    nx.draw_networkx_nodes(G_simple, pos_simple, node_size=node_sizes_simple, node_color='lightblue', alpha=0.8)
    
    # Draw edges with width based on weight
    for u, v, d in G_simple.edges(data=True):
        nx.draw_networkx_edges(G_simple, pos_simple, edgelist=[(u, v)], width=d['weight'] * 0.05, alpha=0.6)
    
    nx.draw_networkx_labels(G_simple, pos_simple, font_size=12, font_family='sans-serif', font_weight='bold')
    
    plt.title('Simplified Keyword Co-occurrence Network (Top Terms Only)', fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/keyword_network_simplified.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Export keyword data
    keyword_df = pd.DataFrame(most_common, columns=['Keyword', 'Frequency'])
    keyword_df.to_csv(f'{RESULTS_PATH}/keyword_frequency_data.csv', index=False)
    
    # Export co-occurrence matrix for the most common keywords
    cooc_df = pd.DataFrame(co_occurrence, index=top_keywords, columns=top_keywords)
    cooc_df.to_csv(f'{RESULTS_PATH}/keyword_cooccurrence_matrix.csv')
    
    return keyword_df

In [9]:
def analyze_topics_lda(df):
    """Perform LDA topic modeling on abstracts"""
    print("Performing LDA topic analysis...")
    
    # Ensure we have processed abstracts
    if 'processed_abstract' not in df.columns:
        df['processed_abstract'] = df['Abstract'].apply(preprocess_text)
    
    # Create document-term matrix
    print("Creating document-term matrix...")
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)
    dtm = vectorizer.fit_transform(df['processed_abstract'])
    
    # Get feature names
    feature_names = vectorizer.get_feature_names_out()
    
    # Perform LDA with 4 topics
    print("Fitting LDA model with 4 topics...")
    lda_model = LatentDirichletAllocation(
        n_components=4,
        random_state=42,
        max_iter=20,
        learning_method='online'
    )
    
    lda_output = lda_model.fit_transform(dtm)
    
    # Assign dominant topic to each document
    dominant_topic = np.argmax(lda_output, axis=1)
    df['dominant_topic'] = dominant_topic
    
    # Get top words for each topic
    def get_top_words(model, feature_names, n_top_words):
        topic_words = []
        for topic_idx, topic in enumerate(model.components_):
            top_words_idx = topic.argsort()[:-n_top_words-1:-1]
            top_words = [feature_names[i] for i in top_words_idx]
            topic_words.append((topic_idx, top_words))
        return topic_words
    
    top_words = get_top_words(lda_model, feature_names, 15)
    
    # Create topic labels based on top words
    topic_labels = {}
    for topic_idx, words in top_words:
        topic_labels[topic_idx] = f"Topic {topic_idx+1}: {', '.join(words[:3])}"
    
    # Visualize topics with descriptive titles
    fig, axes = plt.subplots(2, 2, figsize=(16, 10), sharey=True)
    axes = axes.flatten()
    
    # Plot top words for each topic
    for i, (topic_idx, words) in enumerate(top_words):
        word_importance = [lda_model.components_[topic_idx][feature_names.tolist().index(word)] for word in words]
        ax = axes[i]
        y_pos = np.arange(len(words))
        ax.barh(y_pos, word_importance, align='center')
        ax.set_yticks(y_pos)
        ax.set_yticklabels(words)
        ax.invert_yaxis()
        # Use descriptive title with top 3 words
        ax.set_title(topic_labels[topic_idx], fontsize=12)
        ax.set_xlabel('Importance')
    
    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/lda_topics.png', dpi=300)
    plt.close()
    
    # Create topic distribution visualization
    topic_counts = df['dominant_topic'].value_counts().sort_index()
    
    # Create topic label list for x-axis (preserving order)
    x_labels = [topic_labels[i] for i in range(len(topic_counts))]
    
    plt.figure(figsize=(14, 6))
    bars = plt.bar(
        range(len(topic_counts)), 
        topic_counts.values, 
        color=sns.color_palette("husl", len(topic_counts))
    )
    
    plt.title('Distribution of Documents Across LDA Topics', fontsize=16)
    plt.xlabel('Topic', fontsize=14)
    plt.ylabel('Number of Documents', fontsize=14)
    plt.xticks(range(len(topic_counts)), x_labels, rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add data labels to the top of each bar
    for i, v in enumerate(topic_counts.values):
        plt.text(i, v + 0.5, str(v), ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/topic_distribution.png', dpi=300)
    plt.close()
    
    # Export document-topic assignment
    topic_assignment = df[['Title', 'year', 'dominant_topic']].copy()
    topic_assignment['topic_label'] = topic_assignment['dominant_topic'].map(
        lambda x: topic_labels[x]
    )
    topic_assignment.to_csv(f'{RESULTS_PATH}/document_topic_assignment.csv', index=False)
    
    # Export top words for each topic
    with open(f'{RESULTS_PATH}/topic_keywords.txt', 'w') as f:
        for topic_idx, words in top_words:
            f.write(f"{topic_labels[topic_idx]}\n")
            f.write(f"All top words: {', '.join(words)}\n\n")
    
    return topic_labels

## Thematic Discovery

In [10]:

# Basic data cleaning
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df = df.dropna(subset=['year', 'Abstract'])
df['year'] = df['year'].astype(int)

print(f"Analyzing {len(df)} documents...")

# Run analyses
analyze_publication_trend(df)
analyze_keywords(df)
analyze_document_types(df)
analyze_topics_lda(df)

print("Analysis complete. Results saved to the 'results' directory.")

Analyzing 465 documents...
Analyzing publication trends...
Analyzing keywords...
Co-occurrence threshold: 39.00901244310939
Network contains 22 nodes and 69 edges
Analyzing document types...
Found 16 unique document types:
  - Case Study
  - Commentary
  - Conference Proceedings
  - Cover Story
  - Dissertation/Thesis
  - Feature
  - General Information
  - Interview
  - Journal Article
  - Letter
  - News
  - PERIODICAL
  - Report
  - Review
  - article
  - statistics
Limiting pie chart to top 10 of 16 document types
Document type analysis complete. Found 16 document types.
Performing LDA topic analysis...
Creating document-term matrix...
Fitting LDA model with 4 topics...
Analysis complete. Results saved to the 'results' directory.


## Theme Sorting

In [11]:
def analyze_thematic_categorization(df, themes):
    """
    Categorize papers into predefined themes using keyword matching.
    The function then analyzes distribution and trends over time for these themes.
    Now outputs separate XLSX files for each theme.
    """
    print("Categorizing papers into the three main themes...")
    
    # Ensure we have processed abstracts
    if 'processed_abstract' not in df.columns:
        df['processed_abstract'] = df['Abstract'].apply(preprocess_text)
    
    
    # Function to match document to themes
    def categorize_document(abstract, title):
        if pd.isna(abstract) and pd.isna(title):
            return None
        
        # Combine and preprocess text
        text = ''
        if not pd.isna(abstract):
            text += abstract.lower() + ' '
        if not pd.isna(title):
            text += title.lower()
        
        # Count matches for each theme
        theme_scores = {}
        for theme_name, keywords in themes.items():
            # Count occurrences of each keyword
            score = sum(1 for keyword in keywords if keyword in text)
            theme_scores[theme_name] = score
        
        # Assign primary theme (highest score)
        if max(theme_scores.values()) > 0:
            primary_theme = max(theme_scores.items(), key=lambda x: x[1])[0]
        else:
            primary_theme = "Uncategorized"
            
        return primary_theme, theme_scores
    
    # Apply categorization
    categorization_results = df.apply(
        lambda row: categorize_document(row['processed_abstract'], row['Title']), 
        axis=1
    )
    
    # Extract primary theme and scores
    df['primary_theme'] = [result[0] if result else "Uncategorized" for result in categorization_results]
    df['theme_scores'] = [result[1] if result else {} for result in categorization_results]
    
    # Count papers by theme
    theme_counts = df['primary_theme'].value_counts()
    theme_percent = (theme_counts / theme_counts.sum() * 100).round(1)
    
    print("\nPapers by theme:")
    for theme, count in theme_counts.items():
        print(f"  {theme}: {count} papers ({theme_percent[theme]}%)")
    
    # Create pie chart
    plt.figure(figsize=(12, 8))
    colors = {'Management and Leadership': 'royalblue', 
              'Process Improvement': 'forestgreen',
              'Continuous Learning': 'darkorange',
              'Uncategorized': 'lightgray'}
    
    theme_colors = [colors.get(theme, 'lightgray') for theme in theme_counts.index]
    wedges, texts, autotexts = plt.pie(
        theme_counts, 
        labels=theme_counts.index, 
        autopct='%1.1f%%',
        shadow=False, 
        startangle=90, 
        explode=[0.05] * len(theme_counts),
        colors=theme_colors
    )
    
    # Enhance text visibility
    for text in texts:
        text.set_fontsize(12)
    for autotext in autotexts:
        autotext.set_fontsize(10)
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    plt.title('Distribution of DoD Lean Six Sigma Papers by Theme', fontsize=16)
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/theme_distribution_pie.png', dpi=300)
    plt.close()
    
    # Create bar chart
    plt.figure(figsize=(12, 6))
    bars = plt.bar(
        theme_counts.index,
        theme_counts.values,
        color=[colors.get(theme, 'lightgray') for theme in theme_counts.index]
    )
    
    plt.title('Distribution of DoD Lean Six Sigma Papers by Theme', fontsize=16)
    plt.xlabel('Theme', fontsize=14)
    plt.ylabel('Number of Papers', fontsize=14)
    plt.xticks(rotation=15, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add data labels
    for i, (count, percent) in enumerate(zip(theme_counts, theme_percent)):
        plt.text(
            i, 
            count + 0.5, 
            f"{count} ({percent}%)", 
            ha='center', 
            fontsize=10,
            fontweight='bold'
        )
    
    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/theme_distribution_bar.png', dpi=300)
    plt.close()
    
    # Generate word clouds for each theme
    for theme in themes.keys():
        if theme not in theme_counts.index:
            continue
            
        # Select papers for this theme
        theme_papers = df[df['primary_theme'] == theme]
        
        if len(theme_papers) == 0:
            continue
        
        # Combine all text for this theme
        theme_text = ' '.join(theme_papers['processed_abstract'].dropna())
        
        if not theme_text.strip():
            continue
        
        # Generate word cloud
        wordcloud = WordCloud(
            width=800, 
            height=400, 
            background_color='white',
            max_words=100, 
            contour_width=3, 
            contour_color='steelblue'
        ).generate(theme_text)
        
        plt.figure(figsize=(12, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(f'Key Terms in {theme} Papers', fontsize=16)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f'{RESULTS_PATH}/wordcloud_{theme.replace(" ", "_")}.png', dpi=300)
        plt.close()
    
    # Analyze theme trends over time
    try:
        # Group by year and theme
        yearly_theme_counts = df.groupby(['year', 'primary_theme']).size().unstack(fill_value=0)
        
        # Plot theme trends
        plt.figure(figsize=(14, 8))
        ax = yearly_theme_counts.plot(
            kind='line', 
            marker='o',
            linewidth=2,
            markersize=6,
            ax=plt.gca(),
            color=[colors.get(theme, 'lightgray') for theme in yearly_theme_counts.columns]
        )
        
        plt.title('Evolution of Research Themes Over Time', fontsize=16)
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Number of Publications', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(title='Theme', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(f'{RESULTS_PATH}/theme_trends_over_time.png', dpi=300)
        plt.close()
        
        # Create stacked area chart
        plt.figure(figsize=(14, 8))
        yearly_theme_counts.plot.area(
            stacked=True, 
            alpha=0.7, 
            ax=plt.gca(),
            color=[colors.get(theme, 'lightgray') for theme in yearly_theme_counts.columns]
        )
        plt.title('Cumulative Growth of Research Themes', fontsize=16)
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Number of Publications', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(title='Theme', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(f'{RESULTS_PATH}/theme_cumulative_growth.png', dpi=300)
        plt.close()
        
        # Calculate percentage distribution over time
        yearly_percentages = yearly_theme_counts.div(yearly_theme_counts.sum(axis=1), axis=0) * 100
        
        plt.figure(figsize=(14, 8))
        yearly_percentages.plot.area(
            stacked=True, 
            alpha=0.7, 
            ax=plt.gca(),
            color=[colors.get(theme, 'lightgray') for theme in yearly_percentages.columns]
        )
        plt.title('Relative Proportion of Research Themes Over Time', fontsize=16)
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Percentage of Publications', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(title='Theme', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(f'{RESULTS_PATH}/theme_percentage_over_time.png', dpi=300)
        plt.close()
    except Exception as e:
        print(f"Error in time trend analysis: {e}")
    
    # Export data
    # Save theme categorization
    df[['Title', 'year', 'primary_theme']].to_csv(f'{RESULTS_PATH}/papers_by_theme.csv', index=False)
    
    # Save theme counts
    theme_df = pd.DataFrame({
        'Theme': theme_counts.index,
        'Count': theme_counts.values,
        'Percentage': theme_percent.values
    })
    theme_df.to_csv(f'{RESULTS_PATH}/theme_distribution.csv', index=False)
    
    # Save yearly theme counts
    if 'yearly_theme_counts' in locals() and not yearly_theme_counts.empty:
        yearly_theme_counts.to_csv(f'{RESULTS_PATH}/theme_counts_by_year.csv')
    
    # Create separate XLSX files for each theme (this is the new part)
    for theme in themes.keys():
        theme_papers = df[df['primary_theme'] == theme]
        if len(theme_papers) > 0:
            # Include more comprehensive information for literature review selection
            output_columns = [
                'Title', 'year', 'Authors', 'Abstract', 
                'Journal', 'Volume', 'Issue', 'DOI', 'URL'
            ]
            
            # Only include columns that exist in the dataframe
            available_columns = [col for col in output_columns if col in theme_papers.columns]
            
            # Create the filename
            filename = f'{RESULTS_PATH}/{theme.replace(" ", "_")}_papers.xlsx'
            
            # Export to Excel with enhanced formatting
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                theme_papers[available_columns].to_excel(writer, index=False, sheet_name=theme[:31])  # Excel sheet names have a 31 character limit
                
                # Get the workbook and the worksheet
                workbook = writer.book
                worksheet = writer.sheets[theme[:31]]
                
                # Format header row (bold, background color)
                for col_num, value in enumerate(available_columns, 1):
                    cell = worksheet.cell(row=1, column=col_num)
                    cell.font = openpyxl.styles.Font(bold=True)
                    cell.fill = openpyxl.styles.PatternFill(
                        start_color='E6E6E6',
                        end_color='E6E6E6',
                        fill_type='solid'
                    )
                
                # Adjust column widths for better readability
                for i, column in enumerate(available_columns):
                    if column == 'Title':
                        worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = 40
                    elif column == 'Abstract':
                        worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = 60
                    elif column == 'Authors':
                        worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = 30
                    else:
                        worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = 15
            
            print(f"Created {filename} with {len(theme_papers)} papers")
    
    # Also create an "Uncategorized" file if needed
    uncategorized_papers = df[df['primary_theme'] == "Uncategorized"]
    if len(uncategorized_papers) > 0:
        # Include more comprehensive information
        output_columns = [
            'Title', 'year', 'Authors', 'Abstract', 
            'Journal', 'Volume', 'Issue', 'DOI', 'URL'
        ]
        
        # Only include columns that exist in the dataframe
        available_columns = [col for col in output_columns if col in uncategorized_papers.columns]
        
        # Create the filename
        filename = f'{RESULTS_PATH}/Uncategorized_papers.xlsx'
        
        # Export to Excel with enhanced formatting
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            uncategorized_papers[available_columns].to_excel(writer, index=False, sheet_name='Uncategorized')
            
            # Get the workbook and the worksheet
            workbook = writer.book
            worksheet = writer.sheets['Uncategorized']
            
            # Format header row (bold, background color)
            for col_num, value in enumerate(available_columns, 1):
                cell = worksheet.cell(row=1, column=col_num)
                cell.font = openpyxl.styles.Font(bold=True)
                cell.fill = openpyxl.styles.PatternFill(
                    start_color='E6E6E6',
                    end_color='E6E6E6',
                    fill_type='solid'
                )
            
            # Adjust column widths for better readability
            for i, column in enumerate(available_columns):
                if column == 'Title':
                    worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = 40
                elif column == 'Abstract':
                    worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = 60
                elif column == 'Authors':
                    worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = 30
                else:
                    worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = 15
        
        print(f"Created {filename} with {len(uncategorized_papers)} papers")
    
    print("Theme analysis complete with separate XLSX files for each theme.")
    return theme_df

# Update the import statements at the beginning of the file to include openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import networkx as nx
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import openpyxl  # Add this import for Excel formatting

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import os

In [ ]:
def categorize_papers_by_theme(df, themes):
    """
    Categorize papers into predefined themes using keyword matching.
    Returns the dataframe with a new 'theme' column.
    """
    print("Categorizing papers into themes...")
    
    # Ensure we have processed abstracts
    if 'processed_abstract' not in df.columns:
        df['processed_abstract'] = df['Abstract'].apply(preprocess_text)
    
    # Function to match document to themes
    def categorize_document(abstract, title):
        if pd.isna(abstract) and pd.isna(title):
            return "Uncategorized"
        
        # Combine and preprocess text
        text = ''
        if not pd.isna(abstract):
            text += abstract.lower() + ' '
        if not pd.isna(title):
            text += title.lower()
        
        # Count matches for each theme
        theme_scores = {}
        for theme_name, keywords in themes.items():
            # Count occurrences of each keyword
            score = sum(1 for keyword in keywords if keyword in text)
            theme_scores[theme_name] = score
        
        # Assign primary theme (highest score)
        if max(theme_scores.values()) > 0:
            primary_theme = max(theme_scores.items(), key=lambda x: x[1])[0]
        else:
            primary_theme = "Uncategorized"
            
        return primary_theme
    
    # Apply categorization
    df['theme'] = df.apply(
        lambda row: categorize_document(row['processed_abstract'], row['Title']), 
        axis=1
    )
    
    # Count papers by theme
    theme_counts = df['theme'].value_counts()
    
    print("\nPapers by theme:")
    for theme, count in theme_counts.items():
        print(f"  {theme}: {count} papers")
    
    return df

def create_theme_visualizations(df, results_path):
    """
    Create visualizations for theme distribution.
    """
    print("Creating theme visualizations...")
    
    # Count papers by theme
    theme_counts = df['theme'].value_counts()
    theme_percent = (theme_counts / theme_counts.sum() * 100).round(1)
    
    # Create pie chart
    plt.figure(figsize=(12, 8))
    colors = {'Management and Leadership': 'royalblue', 
              'Process Improvement': 'forestgreen',
              'Continuous Learning': 'darkorange',
              'Uncategorized': 'lightgray'}
    
    theme_colors = [colors.get(theme, 'lightgray') for theme in theme_counts.index]
    wedges, texts, autotexts = plt.pie(
        theme_counts, 
        labels=theme_counts.index, 
        autopct='%1.1f%%',
        shadow=False, 
        startangle=90, 
        explode=[0.05] * len(theme_counts),
        colors=theme_colors
    )
    
    # Enhance text visibility
    for text in texts:
        text.set_fontsize(12)
    for autotext in autotexts:
        autotext.set_fontsize(10)
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    plt.title('Distribution of Papers by Theme', fontsize=16)
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig(f'{results_path}/theme_distribution_pie.png', dpi=300)
    plt.close()
    
    # Create bar chart
    plt.figure(figsize=(12, 6))
    bars = plt.bar(
        theme_counts.index,
        theme_counts.values,
        color=[colors.get(theme, 'lightgray') for theme in theme_counts.index]
    )
    
    plt.title('Distribution of Papers by Theme', fontsize=16)
    plt.xlabel('Theme', fontsize=14)
    plt.ylabel('Number of Papers', fontsize=14)
    plt.xticks(rotation=15, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add data labels
    for i, (count, percent) in enumerate(zip(theme_counts, theme_percent)):
        plt.text(
            i, 
            count + 0.5, 
            f"{count} ({percent}%)", 
            ha='center', 
            fontsize=10,
            fontweight='bold'
        )
    
    plt.tight_layout()
    plt.savefig(f'{results_path}/theme_distribution_bar.png', dpi=300)
    plt.close()
    
    return theme_counts, theme_percent

def create_theme_word_clouds(df, themes, results_path):
    """
    Generate word clouds for each theme.
    """
    print("Creating theme word clouds...")
    
    # Generate word clouds for each theme
    for theme in themes.keys():
        # Select papers for this theme
        theme_papers = df[df['theme'] == theme]
        
        if len(theme_papers) == 0:
            continue
        
        # Combine all text for this theme
        theme_text = ' '.join(theme_papers['processed_abstract'].dropna())
        
        if not theme_text.strip():
            continue
        
        # Generate word cloud
        wordcloud = WordCloud(
            width=800, 
            height=400, 
            background_color='white',
            max_words=100, 
            contour_width=3, 
            contour_color='steelblue'
        ).generate(theme_text)
        
        plt.figure(figsize=(12, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(f'Key Terms in {theme} Papers', fontsize=16)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f'{results_path}/wordcloud_{theme.replace(" ", "_")}.png', dpi=300)
        plt.close()

def analyze_theme_trends(df, results_path):
    """
    Analyze theme trends over time.
    """
    print("Analyzing theme trends over time...")
    
    try:
        # Define colors
        colors = {'Management and Leadership': 'royalblue', 
                'Process Improvement': 'forestgreen',
                'Continuous Learning': 'darkorange',
                'Uncategorized': 'lightgray'}
        
        # Group by year and theme
        yearly_theme_counts = df.groupby(['year', 'theme']).size().unstack(fill_value=0)
        
        # Plot theme trends
        plt.figure(figsize=(14, 8))
        ax = yearly_theme_counts.plot(
            kind='line', 
            marker='o',
            linewidth=2,
            markersize=6,
            ax=plt.gca(),
            color=[colors.get(theme, 'lightgray') for theme in yearly_theme_counts.columns]
        )
        
        plt.title('Evolution of Research Themes Over Time', fontsize=16)
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Number of Publications', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(title='Theme', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(f'{results_path}/theme_trends_over_time.png', dpi=300)
        plt.close()
        
        # Create stacked area chart
        plt.figure(figsize=(14, 8))
        yearly_theme_counts.plot.area(
            stacked=True, 
            alpha=0.7, 
            ax=plt.gca(),
            color=[colors.get(theme, 'lightgray') for theme in yearly_theme_counts.columns]
        )
        plt.title('Cumulative Growth of Research Themes', fontsize=16)
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Number of Publications', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(title='Theme', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(f'{results_path}/theme_cumulative_growth.png', dpi=300)
        plt.close()
        
        # Calculate percentage distribution over time
        yearly_percentages = yearly_theme_counts.div(yearly_theme_counts.sum(axis=1), axis=0) * 100
        
        plt.figure(figsize=(14, 8))
        yearly_percentages.plot.area(
            stacked=True, 
            alpha=0.7, 
            ax=plt.gca(),
            color=[colors.get(theme, 'lightgray') for theme in yearly_percentages.columns]
        )
        plt.title('Relative Proportion of Research Themes Over Time', fontsize=16)
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('Percentage of Publications', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(title='Theme', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(f'{results_path}/theme_percentage_over_time.png', dpi=300)
        plt.close()
        
        # Save yearly theme counts
        yearly_theme_counts.to_csv(f'{results_path}/theme_counts_by_year.csv')
        
    except Exception as e:
        print(f"Error in time trend analysis: {e}")

def export_themed_excel_files(df, results_path):
    """
    Export separate Excel files for each theme using the original dataframe.
    This preserves all columns including URLs and other metadata.
    """
    print("Exporting themed Excel files...")
    
    # Get unique themes
    themes = df['theme'].unique()
    
    for theme in themes:
        # Filter data for this theme
        theme_papers = df[df['theme'] == theme]
        
        if len(theme_papers) == 0:
            continue
        
        # Create filename with sanitized theme name
        filename = f'{results_path}/{theme.replace(" ", "_")}_papers.xlsx'
        
        # Export to Excel with enhanced formatting
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            # Export all columns from the original data
            theme_papers.to_excel(writer, index=False, sheet_name=theme[:31])  # Excel sheet names have 31 char limit
            
            # Get the workbook and worksheet
            workbook = writer.book
            worksheet = writer.sheets[theme[:31]]
            
            # Format header row
            for col_num, column_name in enumerate(theme_papers.columns, 1):
                cell = worksheet.cell(row=1, column=col_num)
                cell.font = Font(bold=True)
                cell.fill = PatternFill(
                    start_color='E6E6E6',
                    end_color='E6E6E6',
                    fill_type='solid'
                )
            
            # Adjust column widths
            for i, column in enumerate(theme_papers.columns):
                col_letter = get_column_letter(i+1)
                if column in ['Title', 'Abstract']:
                    worksheet.column_dimensions[col_letter].width = 50
                elif column == 'Authors':
                    worksheet.column_dimensions[col_letter].width = 30
                elif column in ['URL', 'DOI']:
                    worksheet.column_dimensions[col_letter].width = 40
                else:
                    worksheet.column_dimensions[col_letter].width = 15
        
        print(f"Created {filename} with {len(theme_papers)} papers")

In [12]:
"""
   Categorize papers into the three main themes identified from preliminary analysis:
    1. Management and Leadership
    2. Process Improvement
    3. Continuous Learning (Data, Hansei)
"""
themes = {
    'Management and Leadership': [
        'management', 'leadership', 'project', 'program', 'service', 'officer', 
        'employee', 'organization', 'leader', 'command', 'government', 'agency',
        'strategic', 'executive', 'administrative', 'planning', 'manager', 'performance',
        'organizational', 'effectiveness', 'decision'
    ],
    
    'Process Improvement': [
        'process', 'improvement', 'system', 'model', 'lean', 'sigma', 'six', 'lss', 
        'waste', 'quality', 'efficiency', 'performance', 'optimization', 'methodology',
        'streamline', 'workflow', 'operation', 'engineering', 'design', 'tool', 'technique',
        'kaizen', 'value', 'stream', 'mapping', 'dmaic'
    ],
    
    'Continuous Learning': [
        'data', 'analysis', 'learning', 'continuous', 'hansei', 'improvement', 'training',
        'education', 'knowledge', 'metric', 'measure', 'assessment', 'indicator', 'evaluation',
        'analytics', 'insights', 'information', 'skill', 'competency', 'development', 'innovation',
        'feedback', 'culture', 'adaptation', 'intelligence'
    ]
}

In [13]:
df = categorize_papers_by_theme(df, themes)


theme_counts, theme_percent = create_theme_visualizations(df, results_path)


create_theme_word_clouds(df, themes, results_path)


analyze_theme_trends(df, results_path)


export_themed_excel_files(df, results_path)

Categorizing papers into the three main themes...

Papers by theme:
  Management and Leadership: 251 papers (54.0%)
  Process Improvement: 167 papers (35.9%)
  Continuous Learning: 43 papers (9.2%)
  Uncategorized: 4 papers (0.9%)
Created results/Management_and_Leadership_papers.xlsx with 251 papers
Created results/Process_Improvement_papers.xlsx with 167 papers
Created results/Continuous_Learning_papers.xlsx with 43 papers
Created results/Uncategorized_papers.xlsx with 4 papers
Theme analysis complete with separate XLSX files for each theme.


,Theme,Count,Percentage
0,Management and Leadership,251,54.0
1,Process Improvement,167,35.9
2,Continuous Learning,43,9.2
3,Uncategorized,4,0.9
